### Things that can happen:
- A question is asked:
    - Button lights up for 1.5s
    - Take user input during this time and save to question variable
    - Turn off button
    - There **may** be a sound clip which needs to be played after the 1.5s
- A sound is played:
    - Play sound, possibly depending on answers to previous questions
- Fight mode start:
    - Turn on button
    - Set 1 min timer to trigger fight end
    - Start count for how many times the player has punched
    - At 10s, 20s, 28s, play sound if no punches
    - If number of punches is 7, trigger end fight mode early
- Punch (only during fight mode):
    - Add 1 to number of punches
    - Play sound depending on number of punches
    - Turn off button for a number of seconds depending on punch number
    - Do not add to number of punches during this time
- Fight mode end:
    - Turn off button
    - Play sound depending on number of punches
    - (Maybe reset `start_time`?)

### To do:

Issues:

- If you answered no to Q3, there is a slight overlap with the sounds while you fall asleep
- If you don't go back to the room, the death scene happens too early

Ideas:

- Try allowing two long sounds at once again
- Find limits for simultanious sounds
- Check how long it takes to load a sound in (might be worth pre-loading)
- Use something other than pygame for timing (might be more accurate)
- Have a user object with attributes such as gender, punches, certain question responses etc


In [ ]:
import pygame
import glob
import vlc
# import wiringpi
import time
    
'''
Issues:

- If you answered no to Q3, there is a slight overlap with the sounds while you fall asleep
- If you don't go back to the room, the death scene happens too early

Ideas:

- Try allowing two long sounds at once again
- Find limits for simultanious sounds
- Check how long it takes to load a sound in (might be worth pre-loading)
- Use something other than pygame for timing (might be more accurate)
- Have a user object with attributes such as gender, punches, certain question responses etc
'''

def get_sound_name_from_address(address):
    """
    :param address: address to audio file
    """
    return address.split('/')[1].rstrip('.wav')

def get_sound_files_from_folder(folder):
    """
    :param folder: address of folder containing audio files
    """
    addresses = glob.glob(folder+"*.wav")
    file_dict = {}
    for address in addresses:
        file_dict[get_sound_name_from_address(address)] = address
    return file_dict
    
def merge_dicts(dict1, dict2):
    return(dict2.update(dict1))

# Initialise pygame
pygame.init() 
screen = pygame.display.set_mode((600, 400))

# initialise wiringpi
# wiringpi.wiringPiSetup()

# define GPIO mode
# GPIO23 = 4
# GPIO24 = 5
# GPIO25 = 6
# GPIO26 = 25
# LOW = 0
# HIGH = 1
# OUTPUT = 1
# INPUT = 0
# PULL_DOWN = 1
# wiringpi.pinMode(GPIO23, OUTPUT) # button LED
# wiringpi.pinMode(GPIO25, OUTPUT) # water splash LED
# wiringpi.pinMode(GPIO26, OUTPUT) # fan
# wiringpi.pinMode(GPIO24, INPUT)  # push button
# wiringpi.pullUpDnControl(GPIO24, PULL_DOWN) # pull down

# Define colours
RED = (255, 0, 0)
BLACK = (0, 0, 0)

# Define lengths of time
BUTTON_WAIT = '0:01.5'

# Define number of previous short sounds loaded at once
SHORT_SOUNDS_LOADED = 0

# Define events
button_on_event = pygame.USEREVENT
button_off_event = pygame.USEREVENT + 1
sound_event = pygame.USEREVENT + 2
next_event = pygame.USEREVENT + 3
fight_start_event = pygame.USEREVENT + 4
fight_end_event = pygame.USEREVENT + 5

# Load sounds
sound_files = get_sound_files_from_folder(folder = '4 Channel Bounces/')
merge_dicts(get_sound_files_from_folder(folder = '4 Channel Fight/'), sound_files)

# Define start_time
start_time = pygame.time.get_ticks()

# Define players list with players[0] being short sounds and players[1] being long
players = [[],[]]
short_players = players[0]
long_players = players[1]


# Stores answers to each question with a buffer for Q[0] so the numbers match
Q = [0, False, False, False, False, False]

def play_sound(sound):
#     while len(short_players) >= 1:
#         stop_sound(short_players[0])
#         short_players.pop(0)
    print("Loading", sound)
    instance = vlc.Instance()         # Create new instance
    media = instance.media_new(sound) # Load media file to instance
    player = vlc.MediaPlayer()        # Create players
    player.set_media(media)
    player.play()
    return player

def stop_sound(player):
    player.release()

def mins_to_ticks(time):
    m, s = time.split(':')
    return int(1000 * (int(m) * 60 + float(s)))

# Turn off all GPIO outputs being used
# def turn_all_off():
#     wiringpi.digitalWrite(GPIO23, LOW) # Button light
#     wiringpi.digitalWrite(GPIO25, LOW) # LED
#     wiringpi.digitalWrite(GPIO26, LOW) # Fan

def turn_on_button():
    print('Button on')
#     wiringpi.digitalWrite(GPIO23, HIGH)  

def turn_off_button():
    print('Button off')
#     wiringpi.digitalWrite(GPIO23, LOW)

def wait_until(time):
    time = mins_to_ticks(time)
    now = pygame.time.get_ticks()
    wait = time-(now-start_time)
    return wait

def get_press():
    for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    return True
    return False
#     return wiringpi.digitalRead(GPIO24)

def get_button_response_in_window(window=BUTTON_WAIT, Q=False, button_off=True):
    turn_on_button()
    if type(window) == str: window = mins_to_ticks(window)
    button_start_time = pygame.time.get_ticks()
    print('timer started')
    if button_off: pygame.time.set_timer(button_off_event, window, loops=1)
    pressed = 0
    while (pygame.time.get_ticks() - button_start_time) < window and pressed == 0:
        if get_press():
            Q = True
            pressed = 1
    return Q

def quit_all():
    for player in players:
        stop_sound(player)
    
    
def play_punch_sound(punches):
#     for player in short_players:
#         print('stopping', player)
#         stop_sound(player)
    seventh = False
    if punches == 1:
        short_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
    elif punches == 2:
        short_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
    elif punches == 3:
        short_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
    elif punches == 4:
        short_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
    elif punches == 5:
        short_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
    elif punches >= 6:
        short_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
#     elif punches == 7:
#         pygame.time.set_timer(fight_end_event, 0)
#         short_players.append(play_sound(sound_files['AUDIO_23_PUNCH_2']))
#         end_fight()
#         seventh = True
    return seventh        
        

def get_punches_until(end):
    punches = 0
    ended = False
    while (pygame.time.get_ticks() - start_time) < end and ended == False:
        if get_press():
            turn_off_button()
            punches += 1
            ended = play_punch_sound(punches)
            print(punches, ended)
            time.sleep(2)
            turn_on_button()
            
            
    
def begin_fight():
    turn_on_button()
    fight_start_time = pygame.time.get_ticks()
    pygame.time.set_timer(fight_end_event, mins_to_ticks('1:00'), loops=1)
    punches = 0
    print('Fight started')
    get_punches_until(wait_until('1:02'))
    
def end_fight():
    turn_off_button()
    
    
#=======================================================================================#
# This function contains all events
def perform_event(event_number, Q):
    # Fight at '3:42
    if event_number == 12:
#         stop_sound(long_players[0])
#         long_players.pop(0)
        long_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
        time.sleep(2)
        begin_fight()
        turn_off_button()
        

    return Q, event_number
        
#=======================================================================================#

# First event: Play audio 1
# long_players.append(play_sound(sound_files['AUDIO_1']))
# event_number = 1
# pygame.time.set_timer(next_event, mins_to_ticks('0:13'), loops=1)

# Uncomment this to skip straight to a certain point
event_number = 12
pygame.time.set_timer(next_event, mins_to_ticks('0:1'), loops=1)

while True:
    for event in pygame.event.get():  # For any event (i.e. key press)
        if event.type == pygame.QUIT: # if event is a quit event, (i.e. alt-F4)
            pygame.quit()             # quit game
            quit_all()
        if event.type == next_event:
            print('Event:', event_number)
            Q, event_number = perform_event(event_number, Q)
            event_number += 1
        if event.type == button_off_event:
            turn_off_button()
        if event.type == fight_start_event:
            begin_fight()
        if event.type == fight_end_event:
            end_fight()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_k:
                print('Quitting all')
                quit_all()


In [2]:
import pygame
import glob
import vlc
# import wiringpi
import time
    
'''
Issues:

- If you answered no to Q3, there is a slight overlap with the sounds while you fall asleep
- If you don't go back to the room, the death scene happens too early

Ideas:

- Try allowing two long sounds at once again
- Find limits for simultanious sounds
- Check how long it takes to load a sound in (might be worth pre-loading)
- Use something other than pygame for timing (might be more accurate)
- Have a user object with attributes such as gender, punches, certain question responses etc
'''

def get_sound_name_from_address(address):
    """
    :param address: address to audio file
    """
    return address.split('/')[1].rstrip('.wav')

def get_sound_files_from_folder(folder):
    """
    :param folder: address of folder containing audio files
    """
    addresses = glob.glob(folder+"*.wav")
    file_dict = {}
    for address in addresses:
        file_dict[get_sound_name_from_address(address)] = address
    return file_dict
    
def merge_dicts(dict1, dict2):
    return(dict2.update(dict1))

# Initialise pygame
pygame.init() 
screen = pygame.display.set_mode((600, 400))

# initialise wiringpi
# wiringpi.wiringPiSetup()

# define GPIO mode
# GPIO23 = 4
# GPIO24 = 5
# GPIO25 = 6
# GPIO26 = 25
# LOW = 0
# HIGH = 1
# OUTPUT = 1
# INPUT = 0
# PULL_DOWN = 1
# wiringpi.pinMode(GPIO23, OUTPUT) # button LED
# wiringpi.pinMode(GPIO25, OUTPUT) # water splash LED
# wiringpi.pinMode(GPIO26, OUTPUT) # fan
# wiringpi.pinMode(GPIO24, INPUT)  # push button
# wiringpi.pullUpDnControl(GPIO24, PULL_DOWN) # pull down

# Define colours
RED = (255, 0, 0)
BLACK = (0, 0, 0)

# Define lengths of time
BUTTON_WAIT = '0:01.5'

# Define number of previous short sounds loaded at once
SHORT_SOUNDS_LOADED = 0

# Define events
button_on_event = pygame.USEREVENT
button_off_event = pygame.USEREVENT + 1
sound_event = pygame.USEREVENT + 2
next_event = pygame.USEREVENT + 3
fight_start_event = pygame.USEREVENT + 4
fight_end_event = pygame.USEREVENT + 5

# Load sounds
sound_files = get_sound_files_from_folder(folder = '4 Channel Bounces/')
merge_dicts(get_sound_files_from_folder(folder = '4 Channel Fight/'), sound_files)

# Define start_time
start_time = pygame.time.get_ticks()

# Define players list with players[0] being short sounds and players[1] being long
players = [[],[]]
short_players = players[0]
long_players = players[1]


# Stores answers to each question with a buffer for Q[0] so the numbers match
Q = [0, False, False, False, False, False]

def play_sound(sound):
    while len(short_players) >= 1:
        stop_sound(short_players[0])
        short_players.pop(0)
    print("Loading", sound)
    instance = vlc.Instance()         # Create new instance
    media = instance.media_new(sound) # Load media file to instance
    player = vlc.MediaPlayer()        # Create players
    player.set_media(media)
    player.play()
    return player

def stop_sound(player):
    print(player.get_media())
    print(player.get_media())
    print(player.get_media().release())
    print(player.get_instance().release())
    print(player.release())
    print(player.get_media())
    

def mins_to_ticks(time):
    m, s = time.split(':')
    return int(1000 * (int(m) * 60 + float(s)))

# Turn off all GPIO outputs being used
# def turn_all_off():
#     wiringpi.digitalWrite(GPIO23, LOW) # Button light
#     wiringpi.digitalWrite(GPIO25, LOW) # LED
#     wiringpi.digitalWrite(GPIO26, LOW) # Fan

def turn_on_button():
    print('Button on')
#     wiringpi.digitalWrite(GPIO23, HIGH)  

def turn_off_button():
    print('Button off')
#     wiringpi.digitalWrite(GPIO23, LOW)

def wait_until(time):
    time = mins_to_ticks(time)
    now = pygame.time.get_ticks()
    wait = time-(now-start_time)
    return wait

def get_press():
    for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    return True
    return False
#     return wiringpi.digitalRead(GPIO24)

def get_button_response_in_window(window=BUTTON_WAIT, Q=False, button_off=True):
    turn_on_button()
    if type(window) == str: window = mins_to_ticks(window)
    button_start_time = pygame.time.get_ticks()
    print('timer started')
    if button_off: pygame.time.set_timer(button_off_event, window, loops=1)
    pressed = 0
    while (pygame.time.get_ticks() - button_start_time) < window and pressed == 0:
        if get_press():
            Q = True
            pressed = 1
    return Q

def quit_all():
    while len(short_players) > 0:
        stop_sound(short_players[0])
        short_players.pop(0)
    while len(long_players) > 0:
        stop_sound(long_players[0])
        long_players.pop(0)
    print('all quit')
    
    
def play_punch_sound(punches):
#     for player in short_players:
#         print('stopping', player)
#         stop_sound(player)
    seventh = False
    if punches == 1:
        short_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
    elif punches == 2:
        short_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
    elif punches == 3:
        short_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
    elif punches == 4:
        short_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
    elif punches == 5:
        short_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
    elif punches >= 6:
        short_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
#     elif punches == 7:
#         pygame.time.set_timer(fight_end_event, 0)
#         short_players.append(play_sound(sound_files['AUDIO_23_PUNCH_2']))
#         end_fight()
#         seventh = True
    return seventh        
        

def get_punches_until(end):
    punches = 0
    ended = False
    while (pygame.time.get_ticks() - start_time) < end and ended == False:
        if get_press():
            turn_off_button()
            punches += 1
            ended = play_punch_sound(punches)
            print(punches, ended)
            time.sleep(2)
            turn_on_button()
            
            
    
def begin_fight():
    turn_on_button()
    fight_start_time = pygame.time.get_ticks()
    pygame.time.set_timer(fight_end_event, mins_to_ticks('1:00'), loops=1)
    punches = 0
    print('Fight started')
    get_punches_until(wait_until('1:02'))
    
def end_fight():
    turn_off_button()
    
    
#=======================================================================================#
# This function contains all events
def perform_event(event_number, Q):
    # Fight at '3:42
    if event_number == 12:
#         stop_sound(long_players[0])
#         long_players.pop(0)
        short_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
        time.sleep(2)
        begin_fight()
        turn_off_button()
        

    return Q, event_number
        
#=======================================================================================#

# First event: Play audio 1
# long_players.append(play_sound(sound_files['AUDIO_1']))
# event_number = 1
# pygame.time.set_timer(next_event, mins_to_ticks('0:13'), loops=1)

# Uncomment this to skip straight to a certain point
event_number = 12
pygame.time.set_timer(next_event, mins_to_ticks('0:1'), loops=1)

long_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
presses = 0
while True:
    for event in pygame.event.get():  # For any event (i.e. key press)
        if event.type == pygame.QUIT: # if event is a quit event, (i.e. alt-F4)
            pygame.quit()             # quit game
            quit_all()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_k:
                print('Quitting all')
                presses = 0
                quit_all()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
                short_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
                presses += 1
                print(len(short_players), presses, 'presses')


Loading 4 Channel Fight/AUDIO_16_FIGHT_BASE.wav
Loading 4 Channel Fight/AUDIO_16_FIGHT_BASE.wav
1 1 presses
None
None
None
Loading 4 Channel Fight/AUDIO_16_FIGHT_BASE.wav
1 2 presses
None
None
None
Loading 4 Channel Fight/AUDIO_16_FIGHT_BASE.wav
1 3 presses
None
None
None
None
None
None
all quit


error: video system not initialized

In [1]:
print(len(short_players))

NameError: name 'short_players' is not defined

In [4]:
import time
import vlc
def PlaySong():
    p = vlc.MediaPlayer('4 Channel Fight/AUDIO_16_FIGHT_BASE.wav')
    p.play()
    return p
    p.release()
    

    PlaySong()

In [1]:
import tracemalloc
import pygame
import vlc
import glob

def get_sound_name_from_address(address):
    """
    :param address: address to audio file
    """
    return address.split('/')[1].rstrip('.wav')

def get_sound_files_from_folder(folder):
    """
    :param folder: address of folder containing audio files
    """
    addresses = glob.glob(folder+"*.wav")
    file_dict = {}
    for address in addresses:
        file_dict[get_sound_name_from_address(address)] = address
    return file_dict
    
def merge_dicts(dict1, dict2):
    return(dict2.update(dict1))

# Initialise pygame
pygame.init() 
screen = pygame.display.set_mode((600, 400))

tracemalloc.start()

# Load sounds
sound_files = get_sound_files_from_folder(folder = '4 Channel Bounces/')
merge_dicts(get_sound_files_from_folder(folder = '4 Channel Fight/'), sound_files)

# Define players list with players[0] being short sounds and players[1] being long
players = [[],[]]
short_players = players[0]
long_players = players[1]

playing_mem = []
released_mem = []

def play_sound(sound):
    while len(short_players) >= 1:
        stop_sound(short_players[0])
        short_players.pop(0)
    print("Loading", sound)
    instance = vlc.Instance()         # Create new instance
    media = instance.media_new(sound) # Load media file to instance
    player = vlc.MediaPlayer()        # Create players
    player.set_media(media)
    player.play()
    playing_mem.append(tracemalloc.get_traced_memory()[0])
    return player

def stop_sound(player):
    player.get_media().release()
    player.get_instance().release()
    player.release()
    released_mem.append(tracemalloc.get_traced_memory()[0])
    
def quit_all():
    while len(short_players) > 0:
        stop_sound(short_players[0])
        short_players.pop(0)
    while len(long_players) > 0:
        stop_sound(long_players[0])
        long_players.pop(0)
    print('all quit')
    
presses = 0

start_mem = tracemalloc.get_traced_memory()
while True:
    for event in pygame.event.get():  # For any event (i.e. key press)
        if event.type == pygame.QUIT: # if event is a quit event, (i.e. alt-F4)
            quit_all()
            print(tracemalloc.get_traced_memory())
            tracemalloc.stop()
            pygame.quit() 
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_k:
                print('Quitting all')
                presses = 0
                quit_all()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_SPACE:
#                 short_players.append(play_sound(sound_files['AUDIO_16_FIGHT_BASE']))
                presses += 1
                print(len(short_players), presses, 'presses')


pygame 2.1.2 (SDL 2.0.16, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
0 1 presses
0 2 presses
0 3 presses
0 4 presses
0 5 presses
0 6 presses
0 7 presses
0 8 presses
0 9 presses
0 10 presses
0 11 presses
0 12 presses
0 13 presses
0 14 presses
0 15 presses
0 16 presses
0 17 presses
0 18 presses
0 19 presses
0 20 presses
0 21 presses
0 22 presses
0 23 presses
0 24 presses
0 25 presses
0 26 presses
all quit
(14203, 27214)


error: video system not initialized

In [3]:
sudo apt-get install libopenal-dev

SyntaxError: invalid syntax (<ipython-input-3-0986c85c308b>, line 1)

In [2]:
import numpy as np

print(start_mem)
released = np.array(released_mem)
playing = np.array(playing_mem)

print('Memory difference after releasing', released-playing)
print('Memory difference after playing', playing-released)

print(released)

(10898, 24790)
Memory difference after releasing []
Memory difference after playing []
[]


In [4]:
import numpy as np

print(start_mem)
released = np.array(released_mem)
playing = np.array(playing_mem)

print('Memory difference after releasing', released-playing)
print('Memory difference after playing', playing-released)

print(released)

(10898, 24790)
Memory difference after releasing [16202  -628  -628  -628  -596  -684  -684  -684  -564  -684  -684  -684
  -740  -684  -628  -740  -612  -628  -684  -684  -684  -684  -684  -684
  -684  -604  -740  -684  -628  -684  -684  -684  -684  -684  -684  -596
  -684  -684  -684  -740  -628  -740  -684  -628  -684  -740  -532  -684
  -684  -797]
Memory difference after playing [-16202    628    628    628    596    684    684    684    564    684
    684    684    740    684    628    740    612    628    684    684
    684    684    684    684    684    604    740    684    628    684
    684    684    684    684    684    596    684    684    684    740
    628    740    684    628    684    740    532    684    684    797]
[57133 57837 57957 58069 58678 58766 58822 58878 59054 59174 59230 59286
 59286 59342 59454 59454 59582 59766 59822 59878 59934 59990 60046 60102
 60158 60294 60374 60430 60542 60598 60654 60710 60766 60822 60878 61022
 61166 61222 61278 61585 61697 61697 6

In [9]:
# Not releasing media

import numpy as np

print(start_mem)
released = np.array(released_mem)
playing = np.array(playing_mem)

print(released - playing)

(8929, 22849)
[-246 -628 -628 -628 -596 -628 -628 -628 -564 -628 -628 -628 -628 -628
 -628 -628 -556 -628 -321 -628 -628 -628 -628 -628 -628 -548 -628 -310
 -628 -628 -685]


In [ ]:
start = 11044

playing = [41181, 45952, 46481, 46537, 46537, 46537
release = [44697

In [7]:
# importing the module
import tracemalloc
 
# code or function for which memory
# has to be monitored
def app():
    lt = []
    for i in range(0, 100000):
        lt.append(i)
 
# starting the monitoring
tracemalloc.start()
 
# function call
app()
 
# displaying the memory
print(tracemalloc.get_traced_memory())
 
# stopping the library
tracemalloc.stop()

(1466, 3619104)
